## 本文件代码使用的是灰白图片，并使用拼接实现神经网络的输入

In [279]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from PIL import Image

In [280]:
import os
import random
directory = 'G:/dataset/cat_dog/train/'  # 替换为你的目录路径
filenames = os.listdir(directory)
random.shuffle(filenames)
print(filenames)

['dog.8413.jpg', 'dog.12124.jpg', 'cat.594.jpg', 'cat.7859.jpg', 'dog.9213.jpg', 'cat.219.jpg', 'dog.8998.jpg', 'dog.12043.jpg', 'dog.6704.jpg', 'cat.812.jpg', 'dog.8237.jpg', 'cat.5198.jpg', 'cat.5885.jpg', 'cat.10835.jpg', 'dog.7038.jpg', 'dog.5747.jpg', 'cat.793.jpg', 'cat.3271.jpg', 'cat.5305.jpg', 'cat.4386.jpg', 'dog.1866.jpg', 'cat.1715.jpg', 'dog.7593.jpg', 'cat.10590.jpg', 'dog.7537.jpg', 'cat.10331.jpg', 'dog.12172.jpg', 'dog.7517.jpg', 'dog.12054.jpg', 'dog.10714.jpg', 'cat.10191.jpg', 'cat.3569.jpg', 'dog.6027.jpg', 'dog.5467.jpg', 'dog.5456.jpg', 'dog.8686.jpg', 'cat.10870.jpg', 'cat.8177.jpg', 'dog.11920.jpg', 'cat.6460.jpg', 'cat.128.jpg', 'dog.6986.jpg', 'dog.7238.jpg', 'dog.10660.jpg', 'cat.7233.jpg', 'cat.6726.jpg', 'dog.12222.jpg', 'cat.354.jpg', 'cat.192.jpg', 'dog.10099.jpg', 'cat.11171.jpg', 'dog.9553.jpg', 'dog.9359.jpg', 'cat.4021.jpg', 'dog.8408.jpg', 'cat.7785.jpg', 'dog.10091.jpg', 'cat.3212.jpg', 'cat.2209.jpg', 'dog.9780.jpg', 'dog.9463.jpg', 'cat.7059.jpg'

In [281]:
print(len(filenames))
listcat = [i for i in filenames if i.split('.')[0] == 'cat']
listdog = [i for i in filenames if i.split('.')[0] == 'dog']
print(len(listcat))
print(len(listdog))

17223
9015
8208


In [282]:
class mlpnet(nn.Module):
    def __init__(self):
        super(mlpnet, self).__init__()
        self.fc1 = nn.Linear(300*300, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(64, 2)
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return x
net = mlpnet()
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0005)

In [283]:
# # Load data
# import math
# n = 0
# images = []
# labels = []
# for filename in tqdm(filenames):
#     n+=1
#     image_path = 'G:/dataset/cat_dog/train/'+ filename
#     image = Image.open(image_path)
#     image = image.convert('L')
#     image = image.resize((300, 300),Image.ANTIALIAS)
#     transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5,), (0.5,))
#     ])
#     image = transform(image)
#     image = image.squeeze(0)
#     image = image.reshape(-1)
#     
#     images.append(image)
#     label = image_path.split('/')[-1].split('.')[0]
#     label = 0 if label == 'cat' else 1
#     label = torch.tensor(label)
#     labels.append(label)
#     
#     output = net(image)
#     loss = loss_func(output, label)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()


In [284]:
def testlost(net):
    net.eval()
    filenames_test = os.listdir('G:/dataset/cat_dog/test/')
    random.shuffle(filenames_test)
    wrong = 0
    for filename_test in tqdm(filenames_test):
        image_path = 'G:/dataset/cat_dog/test/'+ filename_test
        image = Image.open(image_path)
        image = image.convert('L')
        image = image.resize((300, 300),Image.ANTIALIAS)
        transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
        ])
        image = transform(image)
        image = image.squeeze(0)
        image = image.reshape(-1)
        
        label = image_path.split('/')[-1].split('.')[0]
        label = 0 if label == 'cat' else 1
        label = torch.tensor(label)
        output = net(image)
        output = torch.argmax(output)
        wrong += (output-label)**2
    print("wrong: "+str(wrong/len(filenames_test)))

In [285]:
# Load data
import math
n = 0
images = []
labels = []
net.train()
for filename in filenames:
    n+=1
    image_path = 'G:/dataset/cat_dog/train/'+ filename
    image = Image.open(image_path)
    image = image.convert('L')
    image = image.resize((300, 300),Image.ANTIALIAS)
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
    ])
    image = transform(image)
    image = image.squeeze(0)
    image = image.reshape(-1)
    
    images.append(image)
    label = image_path.split('/')[-1].split('.')[0]
    label = 0 if label == 'cat' else 1
    labels.append(label)
    


    if n % 30 == 0:        
        images = torch.stack(images)
        labels = torch.tensor(labels)
        outputs = net(images)
        loss = loss_func(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        images = []
        labels = []
    if n%2000==0:
        net.eval()
        testlost(net)
        net.train()



100%|██████████| 2023/2023 [00:17<00:00, 117.22it/s]


wrong: tensor(0.4577)


100%|██████████| 2023/2023 [00:17<00:00, 117.38it/s]


wrong: tensor(0.4231)


100%|██████████| 2023/2023 [00:17<00:00, 117.72it/s]


wrong: tensor(0.4236)


KeyboardInterrupt: 

In [ ]:
# PATH = './model_save/dog_cat.pth'
# torch.save(net.state_dict(), PATH)

In [ ]:
# test_net = mlpnet()
# test_net.load_state_dict(torch.load(PATH))